In [ ]:
%pylab inline

from tqdm.notebook import tqdm
import requests
import pickle
import re
import itertools
import functools
import collections
import string
import time
from bs4 import BeautifulSoup
from sortedcontainers import SortedList # http://www.grantjenks.com/docs/sortedcontainers/sortedlist.html#sortedlist

from adventlib import *

YEAR = 2021
DAY = int('23')

submit1, submit2 = generate_submits(YEAR, DAY)

while True:
  try:
    raw_input = get_input(YEAR, DAY)
    break
  except Exception as e:
    print(e)
    time.sleep(1)

In [ ]:
lines, groups = linesplit("""
#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########
""".strip())

In [ ]:
lines, groups = linesplit(raw_input, verbose=True)

In [ ]:
letters = re.findall(r'[ABCD]', ''.join(lines))
letters

In [ ]:
# l2, l1, m1, m2, m3, r1, r2
# a1, b1, c1, d1
# a2, b2, c2, d2

In [ ]:
lines

In [ ]:
M_lines, _ = linesplit("""
...........
##.#.#.#.##
##.#.#.#.##
""".strip())

In [ ]:
n, m, M = lines_as_matrix_nm(M_lines)
M = M == '.'

In [ ]:
locs = dict(
  l2=(0, 0),
  l1=(0, 1),
  
  m1=(0, 3),
  m2=(0, 5),
  m3=(0, 7),
  
  a1=(1, 2),
  b1=(1, 4),
  c1=(1, 6),
  d1=(1, 8),
  
  a2=(2, 2),
  b2=(2, 4),
  c2=(2, 6),
  d2=(2, 8),
  
  r1=(0, 9),
  r2=(0, 10),
)

In [ ]:
locs_inv = {j: i for i, j in locs.items()}

In [ ]:
def dfs(i, j, visited, prev):
  for di, dj in DIR4:
    ii = i + di
    jj = j + dj
    if 0 <= ii < n and 0 <= jj < m and (ii, jj) not in visited and M[ii, jj]:
      visited.add((ii, jj))
      prev[(ii, jj)] = (i, j)
      dfs(ii, jj, visited, prev)

x = locs['l2']
visited = set([x])
prev = {}
dfs(x[0], x[1], visited, prev)

In [ ]:
prev

In [ ]:
paths = {}
for loc1 in locs:
  x = locs[loc1]
  prev = {}
  dfs(x[0], x[1], set([x]), prev)
  for loc2 in locs:
    if loc1 == loc2:
      continue
    x = locs[loc2]
    path = [x]
    while x != locs[loc1]:
      x = prev[x]
      path.append(x)
    l = len(path) - 1
    path = path[::-1][1:-1]
    
    met = [locs_inv[i] for i in path if i in locs_inv]
    
    paths[(loc1, loc2)] = (l, met)

In [ ]:
paths

In [ ]:
letters

In [ ]:
def key(conf):
  
  # return ''.join(itertools.chain(conf.keys(), conf.values()))
  return repr(tuple(sorted(list(conf.items()))))

In [ ]:
starting_conf = {'a1': 'A',
 'b1': 'C',
 'c1': 'B',
 'd1': 'A',
 'a2': 'D',
 'b2': 'D',
 'c2': 'B',
 'd2': 'C'}
starting_conf

In [ ]:
key(starting_conf)

In [ ]:
moves = {}
for i in itertools.combinations(list(locs.keys()), 8):
  print(i)
  break

In [ ]:
len(locs.keys()), 8

In [ ]:
move_cost_mults = dict(
  A=1,
  B=10,
  C=100,
  D=1000,
)

In [ ]:
def neighbours(conf):
  res = []
  for start_loc in conf:
    for target_loc in locs:
      if start_loc == target_loc or target_loc in conf:
        continue
      pathl, path = paths[start_loc, target_loc]
      
      # collisions?
      bad = False
      for i in path:
        if i in conf:
          bad = True
      if bad:
        continue
        
      me = conf[start_loc]
      color = target_loc[0]
      if color in 'abcd':
        # move to room
        if color != me.lower():
          # not my room
          continue
        if target_loc[1] == '1':
          next_loc = color + '2'
          if next_loc not in conf:
            # a2 is empty, no sense to move to a1
            continue
          if conf[next_loc].lower() != color:
            # a2 is wrong, no sense to move to a1
            continue
      
      if color in 'lmr':
        if start_loc[0] in 'lmr':
          # no hallway-to-hallway
          continue
        if start_loc[0] == me.lower() and start_loc[1] == '2':
          # A doesn't leave a2
          continue
      
      cost = pathl * move_cost_mults[me]
      new_conf = dict(conf)
      del new_conf[start_loc]
      new_conf[target_loc] = me
      res.append((cost, new_conf))
  return res

In [ ]:
dists = {key(starting_conf): 0}
prev = {}
visited = set()
q = SortedList()
q.add((0, key(starting_conf), starting_conf))
dup = 0

with tqdm() as pbar:
  while len(q) > 0:
    dist, _, conf = q.pop(0)
    if rand() < 0.01:
      pbar.set_description(f'dist={dist} q={len(q)} dup={dup}')
    pbar.update()

    if len(q) == 100000:
      break
    if key(conf) in visited:
      dup += 1
      continue
    visited.add(key(conf))

    win = True
    win &= conf.get('a1', '') == 'A'
    win &= conf.get('a2', '') == 'A'
    win &= conf.get('b1', '') == 'B'
    win &= conf.get('b2', '') == 'B'
    win &= conf.get('c1', '') == 'C'
    win &= conf.get('c2', '') == 'C'
    win &= conf.get('d1', '') == 'D'
    win &= conf.get('d2', '') == 'D'
    if win:
      print(dist, conf)
      break

    for dist2, conf2 in neighbours(conf):
      if key(conf2) in visited:
        continue

      if key(conf2) not in dists or dist + dist2 < dists[key(conf2)]:
        dists[key(conf2)] = dist + dist2
        prev[key(conf2)] = conf
        q.add((dists[key(conf2)], key(conf2), conf2))

In [ ]:
submit1(18195)

In [96]:
lines, groups = linesplit("""
#############
#...........#
###A#C#B#A###
  #D#C#B#A#
  #D#B#A#C#
  #D#D#B#C#
  #########
""".strip())

In [97]:
M_lines, _ = linesplit("""
...........
##.#.#.#.##
##.#.#.#.##
##.#.#.#.##
##.#.#.#.##
""".strip())

In [98]:
n, m, M = lines_as_matrix_nm(M_lines)
M = M == '.'

In [100]:
locs = dict(
  l2=(0, 0),
  l1=(0, 1),
  
  m1=(0, 3),
  m2=(0, 5),
  m3=(0, 7),
  
  a1=(1, 2),
  b1=(1, 4),
  c1=(1, 6),
  d1=(1, 8),
  
  a2=(2, 2),
  b2=(2, 4),
  c2=(2, 6),
  d2=(2, 8),
  
  a3=(3, 2),
  b3=(3, 4),
  c3=(3, 6),
  d3=(3, 8),
  
  a4=(4, 2),
  b4=(4, 4),
  c4=(4, 6),
  d4=(4, 8),
  
  r1=(0, 9),
  r2=(0, 10),
)
locs_inv = {j: i for i, j in locs.items()}

In [101]:
def dfs(i, j, visited, prev):
  for di, dj in DIR4:
    ii = i + di
    jj = j + dj
    if 0 <= ii < n and 0 <= jj < m and (ii, jj) not in visited and M[ii, jj]:
      visited.add((ii, jj))
      prev[(ii, jj)] = (i, j)
      dfs(ii, jj, visited, prev)

x = locs['l2']
visited = set([x])
prev = {}
dfs(x[0], x[1], visited, prev)

In [102]:
paths = {}
for loc1 in locs:
  x = locs[loc1]
  prev = {}
  dfs(x[0], x[1], set([x]), prev)
  for loc2 in locs:
    if loc1 == loc2:
      continue
    x = locs[loc2]
    path = [x]
    while x != locs[loc1]:
      x = prev[x]
      path.append(x)
    l = len(path) - 1
    path = path[::-1][1:-1]
    
    met = [locs_inv[i] for i in path if i in locs_inv]
    
    paths[(loc1, loc2)] = (l, met)

In [105]:
def key(conf):
  
  # return ''.join(itertools.chain(conf.keys(), conf.values()))
  return repr(tuple(sorted(list(conf.items()))))

In [106]:
letters = re.findall(r'[ABCD]', ''.join(lines))
letters

['A',
 'C',
 'B',
 'A',
 'D',
 'C',
 'B',
 'A',
 'D',
 'B',
 'A',
 'C',
 'D',
 'D',
 'B',
 'C']

In [108]:
starting_conf = {}
for i, j in enumerate(letters):
  starting_conf[f'{"abcd"[i % 4]}{i//4+1}'] = j
starting_conf

{'a1': 'A',
 'b1': 'C',
 'c1': 'B',
 'd1': 'A',
 'a2': 'D',
 'b2': 'C',
 'c2': 'B',
 'd2': 'A',
 'a3': 'D',
 'b3': 'B',
 'c3': 'A',
 'd3': 'C',
 'a4': 'D',
 'b4': 'D',
 'c4': 'B',
 'd4': 'C'}

In [109]:
move_cost_mults = dict(
  A=1,
  B=10,
  C=100,
  D=1000,
)

In [111]:
def neighbours(conf):
  res = []
  for start_loc in conf:
    for target_loc in locs:
      if start_loc == target_loc or target_loc in conf:
        continue
      pathl, path = paths[start_loc, target_loc]
      
      # collisions?
      bad = False
      for i in path:
        if i in conf:
          bad = True
      if bad:
        continue
        
      me = conf[start_loc]
      color = target_loc[0]
      if color in 'abcd':
        # move to room
        if color != me.lower():
          # not my room
          continue
        if target_loc[1] == '1':
          next_loc = color + '2'
          if next_loc not in conf:
            # a2 is empty, no sense to move to a1
            continue
          if conf[next_loc].lower() != color:
            # a2 is wrong, no sense to move to a1
            continue
        if target_loc[1] == '2':
          next_loc = color + '3'
          if next_loc not in conf:
            # a2 is empty, no sense to move to a1
            continue
          if conf[next_loc].lower() != color:
            # a2 is wrong, no sense to move to a1
            continue
        if target_loc[1] == '3':
          next_loc = color + '4'
          if next_loc not in conf:
            # a2 is empty, no sense to move to a1
            continue
          if conf[next_loc].lower() != color:
            # a2 is wrong, no sense to move to a1
            continue
      
      if color in 'lmr':
        if start_loc[0] in 'lmr':
          # no hallway-to-hallway
          continue
        if start_loc[0] == me.lower() and start_loc[1] == '4':
          # A doesn't leave a4
          continue
      
      cost = pathl * move_cost_mults[me]
      new_conf = dict(conf)
      del new_conf[start_loc]
      new_conf[target_loc] = me
      res.append((cost, new_conf))
  return res

In [116]:
dists = {key(starting_conf): 0}
prev = {}
visited = set()
q = SortedList()
q.add((0, key(starting_conf), starting_conf))
dup = 0

with tqdm() as pbar:
  while len(q) > 0:
    dist, _, conf = q.pop(0)
    if rand() < 0.001:
      pbar.set_description(f'dist={dist} q={len(q)} dup={dup}')
      pbar.update()

    if len(q) == 100000:
      break
    if key(conf) in visited:
      dup += 1
      continue
    visited.add(key(conf))

    win = True
    for i in 'abcd':
      for j in range(1, 5):
        win &= conf.get(f'{i}{j}', '') == i.upper()
    if win:
      print(dist, conf)
      break

    for dist2, conf2 in neighbours(conf):
      if key(conf2) in visited:
        continue

      if key(conf2) not in dists or dist + dist2 < dists[key(conf2)]:
        dists[key(conf2)] = dist + dist2
        prev[key(conf2)] = conf
        q.add((dists[key(conf2)], key(conf2), conf2))

0it [00:00, ?it/s]

50265 {'c4': 'C', 'c3': 'C', 'b4': 'B', 'b3': 'B', 'b2': 'B', 'c2': 'C', 'c1': 'C', 'd4': 'D', 'b1': 'B', 'a4': 'A', 'a3': 'A', 'd3': 'D', 'd2': 'D', 'd1': 'D', 'a2': 'A', 'a1': 'A'}


In [118]:
SUBMITTED_ANSWERS

{(2021, 23, 1, 18195): True,
 (2021,
  23,
  2,
  "(('a1', 'A'), ('a2', 'A'), ('a3', 'A'), ('a4', 'A'), ('b1', 'B'), ('b2', 'B'), ('b3', 'B'), ('b4', 'B'), ('c1', 'C'), ('c2', 'C'), ('c3', 'C'), ('c4', 'C'), ('d1', 'D'), ('d2', 'D'), ('d3', 'D'), ('d4', 'D'))"): False}

In [122]:
submit2(50265)

True